Prueba código para proyecto

In [ ]:
#para cargar archivos de mi computadora
#from google.colab import files
#uploaded = files.upload()

In [ ]:
#para conectarse directamente al drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
import pandas as pd
import json
import time
from datetime import datetime

DEFINIR FUNCIONES

In [ ]:
#API_KEY FUNCTION: returns the api key encripted (it is saved in a txt)
def api_key():
    with open('/content/drive/MyDrive/1. IRONHACK/respaldo proyecto/apiKey.txt') as f:
        contents = f.read()
        api = f'?api_key={contents}'
        return api

In [ ]:
#SUMMONER FUNCTION: returns the first version of the summoners df
def summoner(api):
    #empty list to append all the tables
    main_df = list()

    #defining a list with my 3 top regions
    region_list = ['eun1','kr','na1']

    #defining a list with my 3 top tiers
    tier_list = ['challengerleagues','grandmasterleagues','masterleagues']

    #loop that will change the url so the 9 combinations are appended to the main_df list
    for region in region_list:
        for tier in tier_list:
            url = f'https://{region}.api.riotgames.com/lol/league/v4/{tier}/by-queue/RANKED_SOLO_5x5{api}'
            response_json = requests.get(url)
            response = response_json.json()
            #print(response)
            response_df = pd.DataFrame(response)
            #adding the column "region" (it will be necessary later)
            response_df['region'] = region
            #print(response_df)
            main_df.append(response_df)
            time.sleep(2)
    summoner = pd.concat(main_df)
    return summoner


In [ ]:
#CHAMPION AND SUMMONER FUNCTION: returns the first version of the summoners df with their champions
def championAndSummoner(api,flat_summoners):
    #empty list to append all the tables
    main_df_1 = list()

    #a for loop to consider all the summoners in the flat_summoners_ordered df
    for i in range(len(flat_summoners)):
        #since there are more than 7k rows, to practice, we can consider only 50
        if i > 8350:
            break
        #we want the first value of the list given by the .at[i,region]    
        region = flat_summoners.at[i, 'region'][0]
        invocador = flat_summoners.at[i,'summonerId'][0]

        url = f'https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{invocador}{api}'
        response_json = requests.get(url)
        response = response_json.json()
        response_df = pd.DataFrame(response)
        main_df_1.append(response_df)
        time.sleep(1)
    champion_and_summoner = pd.concat(main_df_1)
    return champion_and_summoner

In [ ]:
#CHAMPION AND SUMMONER FUNCTION: returns the first version of the summoners df with their champions
def champions():
    champs_json = requests.get('http://ddragon.leagueoflegends.com/cdn/10.23.1/data/en_US/champion.json')
    champs = champs_json.json()
    champs_df = pd.DataFrame(champs)
    return champs_df

In [ ]:
#CHANGE_INDEX FUNCTION: function that will change the index of pd
def change_index(df):
    index = df.shape[0]
    index_final = [*range(0, index, 1)]
    df = df.set_axis([index_final], axis='index')
    return df

In [ ]:
#FLATTEN FUNCTION: will flatten my data (if a column has nested dictionaries)
def flatten(table, column_list):
    for column in column_list:
        flattened = pd.DataFrame(dict(table[column])).transpose()
        columns = [str(col) for col in flattened.columns]
        flattened.columns = [column + '_' + column_name for column_name in columns]
        table = pd.concat([table, flattened], axis=1)
        table = table.drop(column, axis = 1)
    return table

In [ ]:
#MERGE FUNCTION: it will "concat" two tables according an specific value (this case championID)
def merge(df1, df2):
    df = df1.merge(df2,left_on='data_key', right_on='championId')
    return df

In [ ]:
#CSV FUNCTION
def convert_csv(df,name):
    df.to_csv(name)

### CLEANING FUNCTIONS

In [ ]:
#ORDER AND RENAME FUNCTION
def order(df,lista):
    #order columns and delete the ones I won't use
    flat = df[(lista)]
    return flat

In [ ]:
def rename(df,dicc):
    flat_ordered = df.rename(columns=dicc)
    return flat_ordered

### FUNCTIONS NESTED

summoners

In [ ]:
api = api_key()
api

'?api_key=RGAPI-27157efd-8530-4475-b46a-592ef8286555'

In [ ]:
summoner_df = summoner(api)
summoner_df

,tier,leagueId,queue,name,entries,region
0,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'WUR7bNRsr-2eIuhH9dtjHyAPqGeDuN...,eun1
1,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'OSB7m-MIa4StpbtrV9bIknv6ZA807a...,eun1
2,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': '3YH8ctLOFi7Y6w1h0TebelymCD-9W6...,eun1
3,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'FqV3JL8aHdhTRuCNSrGaghwZDrK7vb...,eun1
4,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'EILXDFBDb9QivdJDBt9JL0t0r3TEHc...,eun1
...,...,...,...,...,...,...
1139,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,{'summonerId': 'gIAllAqz9DKtlP0OZe7SCA8qu3c6gW...,na1
1140,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,{'summonerId': 'GoiW4k12D1V8bwwDX36F9OBnqA44u0...,na1
1141,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,{'summonerId': '_g1CHkNIr0P04p0-TMWZOFT6uRDPTL...,na1
1142,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,{'summonerId': '0mHy5GxFTNaQinMWMd9K4ZRYWji5SQ...,na1


In [ ]:
summoner_df = change_index(summoner_df)
summoner_df

,tier,leagueId,queue,name,entries,region
0,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'WUR7bNRsr-2eIuhH9dtjHyAPqGeDuN...,eun1
1,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'OSB7m-MIa4StpbtrV9bIknv6ZA807a...,eun1
2,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': '3YH8ctLOFi7Y6w1h0TebelymCD-9W6...,eun1
3,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'FqV3JL8aHdhTRuCNSrGaghwZDrK7vb...,eun1
4,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'EILXDFBDb9QivdJDBt9JL0t0r3TEHc...,eun1
...,...,...,...,...,...,...
8391,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,{'summonerId': 'gIAllAqz9DKtlP0OZe7SCA8qu3c6gW...,na1
8392,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,{'summonerId': 'GoiW4k12D1V8bwwDX36F9OBnqA44u0...,na1
8393,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,{'summonerId': '_g1CHkNIr0P04p0-TMWZOFT6uRDPTL...,na1
8394,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,{'summonerId': '0mHy5GxFTNaQinMWMd9K4ZRYWji5SQ...,na1


In [ ]:
nested_column = ['entries']

flat_summoners = flatten(summoner_df, nested_column)
flat_summoners

,tier,leagueId,queue,name,region,entries_summonerId,entries_summonerName,entries_leaguePoints,entries_rank,entries_wins,entries_losses,entries_veteran,entries_inactive,entries_freshBlood,entries_hotStreak
0,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,eun1,WUR7bNRsr-2eIuhH9dtjHyAPqGeDuNOYI5UuR1Uu9O3Svq0,Twtv KappaBeetle,537,I,389,343,False,False,False,False
1,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,eun1,OSB7m-MIa4StpbtrV9bIknv6ZA807aQDAT45OmpO9iJGsxc,Kizuro,728,I,153,108,False,False,False,True
2,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,eun1,3YH8ctLOFi7Y6w1h0TebelymCD-9W6M1cQkDoWkEt9SFHsA,BES Finn,823,I,112,58,False,False,False,False
3,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,eun1,FqV3JL8aHdhTRuCNSrGaghwZDrK7vbZXywZyLpv89nJdFMk,Sica7 is back,820,I,567,481,True,False,False,False
4,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,eun1,EILXDFBDb9QivdJDBt9JL0t0r3TEHcq8Wednm-9FGn01VoE,Linley Baruch,872,I,206,157,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8391,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,na1,gIAllAqz9DKtlP0OZe7SCA8qu3c6gWfjv687Uhzw8TqP6PY,digitalotus,0,I,182,172,False,False,False,True
8392,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,na1,GoiW4k12D1V8bwwDX36F9OBnqA44u0EIBL-xFp6DaA-Ukq8,I smell magic,0,I,488,437,False,False,True,False
8393,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,na1,_g1CHkNIr0P04p0-TMWZOFT6uRDPTLZx7EvqxofP2zTncKc1,Artoria,0,I,365,279,False,False,False,False
8394,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,RANKED_SOLO_5x5,Varus's Villains,na1,0mHy5GxFTNaQinMWMd9K4ZRYWji5SQOjXMGtPnDJ6fhbvHs,Kelpoof,59,I,334,223,False,False,False,False


In [ ]:
lista = ['entries_summonerId','region','entries_summonerName','entries_rank',
                'entries_wins','entries_losses','tier','leagueId','name','entries_leaguePoints']

flat_summoners = order(flat_summoners, lista)
flat_summoners

,entries_summonerId,region,entries_summonerName,entries_rank,entries_wins,entries_losses,tier,leagueId,name,entries_leaguePoints
0,WUR7bNRsr-2eIuhH9dtjHyAPqGeDuNOYI5UuR1Uu9O3Svq0,eun1,Twtv KappaBeetle,I,389,343,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,537
1,OSB7m-MIa4StpbtrV9bIknv6ZA807aQDAT45OmpO9iJGsxc,eun1,Kizuro,I,153,108,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,728
2,3YH8ctLOFi7Y6w1h0TebelymCD-9W6M1cQkDoWkEt9SFHsA,eun1,BES Finn,I,112,58,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,823
3,FqV3JL8aHdhTRuCNSrGaghwZDrK7vbZXywZyLpv89nJdFMk,eun1,Sica7 is back,I,567,481,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,820
4,EILXDFBDb9QivdJDBt9JL0t0r3TEHcq8Wednm-9FGn01VoE,eun1,Linley Baruch,I,206,157,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,872
...,...,...,...,...,...,...,...,...,...,...
8391,gIAllAqz9DKtlP0OZe7SCA8qu3c6gWfjv687Uhzw8TqP6PY,na1,digitalotus,I,182,172,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,Varus's Villains,0
8392,GoiW4k12D1V8bwwDX36F9OBnqA44u0EIBL-xFp6DaA-Ukq8,na1,I smell magic,I,488,437,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,Varus's Villains,0
8393,_g1CHkNIr0P04p0-TMWZOFT6uRDPTLZx7EvqxofP2zTncKc1,na1,Artoria,I,365,279,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,Varus's Villains,0
8394,0mHy5GxFTNaQinMWMd9K4ZRYWji5SQOjXMGtPnDJ6fhbvHs,na1,Kelpoof,I,334,223,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,Varus's Villains,59


In [ ]:
this_dict ={'entries_summonerId':'summonerId','entries_summonerName':'summonerName', 
            'entries_leaguePoints':'leaguePoints','entries_rank':'rank',
            'entries_wins':'wins','entries_losses':'losses','name':'leagueName'}

flat_summoners = rename(flat_summoners, this_dict)
flat_summoners

,summonerId,region,summonerName,rank,wins,losses,tier,leagueId,leagueName,leaguePoints
0,WUR7bNRsr-2eIuhH9dtjHyAPqGeDuNOYI5UuR1Uu9O3Svq0,eun1,Twtv KappaBeetle,I,389,343,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,537
1,OSB7m-MIa4StpbtrV9bIknv6ZA807aQDAT45OmpO9iJGsxc,eun1,Kizuro,I,153,108,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,728
2,3YH8ctLOFi7Y6w1h0TebelymCD-9W6M1cQkDoWkEt9SFHsA,eun1,BES Finn,I,112,58,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,823
3,FqV3JL8aHdhTRuCNSrGaghwZDrK7vbZXywZyLpv89nJdFMk,eun1,Sica7 is back,I,567,481,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,820
4,EILXDFBDb9QivdJDBt9JL0t0r3TEHcq8Wednm-9FGn01VoE,eun1,Linley Baruch,I,206,157,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,872
...,...,...,...,...,...,...,...,...,...,...
8391,gIAllAqz9DKtlP0OZe7SCA8qu3c6gWfjv687Uhzw8TqP6PY,na1,digitalotus,I,182,172,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,Varus's Villains,0
8392,GoiW4k12D1V8bwwDX36F9OBnqA44u0EIBL-xFp6DaA-Ukq8,na1,I smell magic,I,488,437,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,Varus's Villains,0
8393,_g1CHkNIr0P04p0-TMWZOFT6uRDPTLZx7EvqxofP2zTncKc1,na1,Artoria,I,365,279,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,Varus's Villains,0
8394,0mHy5GxFTNaQinMWMd9K4ZRYWji5SQOjXMGtPnDJ6fhbvHs,na1,Kelpoof,I,334,223,MASTER,136579ee-a212-34b0-b4b8-69ba529f4d7f,Varus's Villains,59


In [ ]:
flat_summoners.dtypes

summonerId      object
region          object
summonerName    object
rank            object
wins            object
losses          object
tier            object
leagueId        object
leagueName      object
leaguePoints    object
dtype: object

In [ ]:
name_csv = 'prueba_1_summoners_output.csv'

summoners_df = convert_csv(flat_summoners,name_csv)

Champions and summoners

In [ ]:
champions_summoner = championAndSummoner(api,flat_summoners)
champions_summoner

,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId,status
0,157.0,7.0,851278.0,1.606147e+12,829678.0,0.0,True,0.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
1,8.0,7.0,276915.0,1.605885e+12,255315.0,0.0,True,0.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
2,67.0,5.0,145378.0,1.606942e+12,123778.0,0.0,True,2.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
3,38.0,7.0,141032.0,1.606991e+12,119432.0,0.0,True,0.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
4,238.0,5.0,108321.0,1.588803e+12,86721.0,0.0,False,2.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
...,...,...,...,...,...,...,...,...,...,...
134,60.0,1.0,189.0,1.557476e+12,189.0,1611.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN
135,50.0,1.0,158.0,1.564264e+12,158.0,1642.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN
136,432.0,1.0,135.0,1.560233e+12,135.0,1665.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN
137,40.0,1.0,105.0,1.578796e+12,105.0,1695.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN


In [ ]:
name_csv = 'prueba_1_champios_and_summoners_output.csv'

champions_summoner_df = convert_csv(champions_summoner,name_csv)

NameError: ignored

### YA SE GUARDARON LOS DOS CSV

In [ ]:
champions_summoner_df = pd.read_csv('/content/drive/MyDrive/1. IRONHACK/respaldo proyecto/BASES CSV/prueba_1_champios_and_summoners_output')
champions_summoner_df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId,status
0,0,157.0,7.0,851278.0,1.606147e+12,829678.0,0.0,True,0.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
1,1,8.0,7.0,276915.0,1.605885e+12,255315.0,0.0,True,0.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
2,2,67.0,5.0,145378.0,1.606942e+12,123778.0,0.0,True,2.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
3,3,38.0,7.0,141032.0,1.606991e+12,119432.0,0.0,True,0.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
4,4,238.0,5.0,108321.0,1.588803e+12,86721.0,0.0,False,2.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
...,...,...,...,...,...,...,...,...,...,...,...
903087,134,60.0,1.0,189.0,1.557476e+12,189.0,1611.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN
903088,135,50.0,1.0,158.0,1.564264e+12,158.0,1642.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN
903089,136,432.0,1.0,135.0,1.560233e+12,135.0,1665.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN
903090,137,40.0,1.0,105.0,1.578796e+12,105.0,1695.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN


In [ ]:
champions_summoner_df = change_index(champions_summoner_df)
champions_summoner_df

,Unnamed: 0,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId,status
0,0,157.0,7.0,851278.0,1.606147e+12,829678.0,0.0,True,0.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
1,1,8.0,7.0,276915.0,1.605885e+12,255315.0,0.0,True,0.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
2,2,67.0,5.0,145378.0,1.606942e+12,123778.0,0.0,True,2.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
3,3,38.0,7.0,141032.0,1.606991e+12,119432.0,0.0,True,0.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
4,4,238.0,5.0,108321.0,1.588803e+12,86721.0,0.0,False,2.0,NSYHCM1YFQvcmgyPy9a6eDRtr3gjOVDqA5BoxY_ZDoUfhYs,NaN
...,...,...,...,...,...,...,...,...,...,...,...
903087,134,60.0,1.0,189.0,1.557476e+12,189.0,1611.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN
903088,135,50.0,1.0,158.0,1.564264e+12,158.0,1642.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN
903089,136,432.0,1.0,135.0,1.560233e+12,135.0,1665.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN
903090,137,40.0,1.0,105.0,1.578796e+12,105.0,1695.0,False,0.0,13KedSMWS-NgjpPLvoMkZa6rMTfna-wJ69X29N_VIwilLO63,NaN


In [ ]:
champions_summoner_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 903092 entries, (0,) to (903091,)
Data columns (total 11 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    903092 non-null  object 
 1   championId                    902828 non-null  float64
 2   championLevel                 902828 non-null  float64
 3   championPoints                902828 non-null  float64
 4   lastPlayTime                  902828 non-null  float64
 5   championPointsSinceLastLevel  902828 non-null  float64
 6   championPointsUntilNextLevel  902828 non-null  float64
 7   chestGranted                  902828 non-null  object 
 8   tokensEarned                  902828 non-null  float64
 9   summonerId                    902828 non-null  object 
 10  status                        264 non-null     object 
dtypes: float64(7), object(4)
memory usage: 126.1+ MB


In [ ]:
#I need to change the values of the column "championId" from int64 to str
champions_summoner_df['championId'] = champions_summoner_df['championId'].astype('str')

In [ ]:
champions_summoner_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 903092 entries, (0,) to (903091,)
Data columns (total 11 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    903092 non-null  object 
 1   championId                    903092 non-null  object 
 2   championLevel                 902828 non-null  float64
 3   championPoints                902828 non-null  float64
 4   lastPlayTime                  902828 non-null  float64
 5   championPointsSinceLastLevel  902828 non-null  float64
 6   championPointsUntilNextLevel  902828 non-null  float64
 7   chestGranted                  902828 non-null  object 
 8   tokensEarned                  902828 non-null  float64
 9   summonerId                    902828 non-null  object 
 10  status                        264 non-null     object 
dtypes: float64(6), object(5)
memory usage: 126.1+ MB


In [ ]:
champions_summoner_df

In [ ]:
name_csv = 'prueba_1_champios_and_summoners_output.csv'

champions_summoner_df = convert_csv(champions_summoner_df,name_csv)

Only champions

In [ ]:
champs = champions()
champs

,type,format,version,data
Aatrox,champion,standAloneComplex,10.23.1,"{'version': '10.23.1', 'id': 'Aatrox', 'key': ..."
Ahri,champion,standAloneComplex,10.23.1,"{'version': '10.23.1', 'id': 'Ahri', 'key': '1..."
Akali,champion,standAloneComplex,10.23.1,"{'version': '10.23.1', 'id': 'Akali', 'key': '..."
Alistar,champion,standAloneComplex,10.23.1,"{'version': '10.23.1', 'id': 'Alistar', 'key':..."
Amumu,champion,standAloneComplex,10.23.1,"{'version': '10.23.1', 'id': 'Amumu', 'key': '..."
...,...,...,...,...
Zed,champion,standAloneComplex,10.23.1,"{'version': '10.23.1', 'id': 'Zed', 'key': '23..."
Ziggs,champion,standAloneComplex,10.23.1,"{'version': '10.23.1', 'id': 'Ziggs', 'key': '..."
Zilean,champion,standAloneComplex,10.23.1,"{'version': '10.23.1', 'id': 'Zilean', 'key': ..."
Zoe,champion,standAloneComplex,10.23.1,"{'version': '10.23.1', 'id': 'Zoe', 'key': '14..."


In [ ]:
nested_column = ['data']

flat_champions = flatten(champs, nested_column)
flat_champions.head(3)

,type,format,version,data_version,data_id,data_key,data_name,data_title,data_blurb,data_info,data_image,data_tags,data_partype,data_stats
Aatrox,champion,standAloneComplex,10.23.1,10.23.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","[Fighter, Tank]",Blood Well,"{'hp': 580, 'hpperlevel': 90, 'mp': 0, 'mpperl..."
Ahri,champion,standAloneComplex,10.23.1,10.23.1,Ahri,103,Ahri,the Nine-Tailed Fox,Innately connected to the latent power of Rune...,"{'attack': 3, 'defense': 4, 'magic': 8, 'diffi...","{'full': 'Ahri.png', 'sprite': 'champion0.png'...","[Mage, Assassin]",Mana,"{'hp': 526, 'hpperlevel': 92, 'mp': 418, 'mppe..."
Akali,champion,standAloneComplex,10.23.1,10.23.1,Akali,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,"{'attack': 5, 'defense': 3, 'magic': 8, 'diffi...","{'full': 'Akali.png', 'sprite': 'champion0.png...",[Assassin],Energy,"{'hp': 575, 'hpperlevel': 95, 'mp': 200, 'mppe..."


In [ ]:
nested_column = ['data_info']

flat_champions = flatten(flat_champions, nested_column)
flat_champions.head(3)

,type,format,version,data_version,data_id,data_key,data_name,data_title,data_blurb,data_image,data_tags,data_partype,data_stats,data_info_attack,data_info_defense,data_info_magic,data_info_difficulty
Aatrox,champion,standAloneComplex,10.23.1,10.23.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","[Fighter, Tank]",Blood Well,"{'hp': 580, 'hpperlevel': 90, 'mp': 0, 'mpperl...",8,4,3,4
Ahri,champion,standAloneComplex,10.23.1,10.23.1,Ahri,103,Ahri,the Nine-Tailed Fox,Innately connected to the latent power of Rune...,"{'full': 'Ahri.png', 'sprite': 'champion0.png'...","[Mage, Assassin]",Mana,"{'hp': 526, 'hpperlevel': 92, 'mp': 418, 'mppe...",3,4,8,5
Akali,champion,standAloneComplex,10.23.1,10.23.1,Akali,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,"{'full': 'Akali.png', 'sprite': 'champion0.png...",[Assassin],Energy,"{'hp': 575, 'hpperlevel': 95, 'mp': 200, 'mppe...",5,3,8,7


In [ ]:
nested_column = ['data_stats']

flat_champions = flatten(flat_champions, nested_column)
flat_champions.head(3)

,type,format,version,data_version,data_id,data_key,data_name,data_title,data_blurb,data_image,data_tags,data_partype,data_info_attack,data_info_defense,data_info_magic,data_info_difficulty,data_stats_hp,data_stats_hpperlevel,data_stats_mp,data_stats_mpperlevel,data_stats_movespeed,data_stats_armor,data_stats_armorperlevel,data_stats_spellblock,data_stats_spellblockperlevel,data_stats_attackrange,data_stats_hpregen,data_stats_hpregenperlevel,data_stats_mpregen,data_stats_mpregenperlevel,data_stats_crit,data_stats_critperlevel,data_stats_attackdamage,data_stats_attackdamageperlevel,data_stats_attackspeedperlevel,data_stats_attackspeed
Aatrox,champion,standAloneComplex,10.23.1,10.23.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","[Fighter, Tank]",Blood Well,8,4,3,4,580.0,90.0,0.0,0.0,345.0,38.00,3.25,32.1,1.25,175.0,3.0,1.0,0.0,0.0,0.0,0.0,60.00,5.0,2.5,0.651
Ahri,champion,standAloneComplex,10.23.1,10.23.1,Ahri,103,Ahri,the Nine-Tailed Fox,Innately connected to the latent power of Rune...,"{'full': 'Ahri.png', 'sprite': 'champion0.png'...","[Mage, Assassin]",Mana,3,4,8,5,526.0,92.0,418.0,25.0,330.0,20.88,3.50,30.0,0.50,550.0,5.5,0.6,8.0,0.8,0.0,0.0,53.04,3.0,2.0,0.668
Akali,champion,standAloneComplex,10.23.1,10.23.1,Akali,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,"{'full': 'Akali.png', 'sprite': 'champion0.png...",[Assassin],Energy,5,3,8,7,575.0,95.0,200.0,0.0,345.0,23.00,3.50,37.0,1.25,125.0,8.0,0.5,50.0,0.0,0.0,0.0,62.40,3.3,3.2,0.625


In [ ]:
flat_champions['data_tags']

Aatrox      [Fighter, Tank]
Ahri       [Mage, Assassin]
Akali            [Assassin]
Alistar     [Tank, Support]
Amumu          [Tank, Mage]
                 ...       
Zed              [Assassin]
Ziggs                [Mage]
Zilean      [Support, Mage]
Zoe         [Mage, Support]
Zyra        [Mage, Support]
Name: data_tags, Length: 152, dtype: object

In [ ]:
flat_champions['data_partype'].unique()

array(['Blood Well', 'Mana', 'Energy', 'None', 'Rage', 'Courage',
       'Shield', 'Fury', 'Ferocity', 'Heat', 'Grit', 'Crimson Rush',
       'Flow'], dtype=object)

In [ ]:
#cada champion tiene 1 o 2 clasificaciones, ambas vienen en formato de lista en la columna "data_tags"
#vamos a expandir las listas que estan en la columna "data_tags" con el metodo aply
tags = flat_champions['data_tags'].apply(pd.Series)
tags

,0,1
Aatrox,Fighter,Tank
Ahri,Mage,Assassin
Akali,Assassin,NaN
Alistar,Tank,Support
Amumu,Tank,Mage
...,...,...
Zed,Assassin,NaN
Ziggs,Mage,NaN
Zilean,Support,Mage
Zoe,Mage,Support


In [ ]:
tags2 = tags.rename(columns={0:'Champion_class', 1:'Champion_class_2'})
tags2

,Champion_class,Champion_class_2
Aatrox,Fighter,Tank
Ahri,Mage,Assassin
Akali,Assassin,NaN
Alistar,Tank,Support
Amumu,Tank,Mage
...,...,...
Zed,Assassin,NaN
Ziggs,Mage,NaN
Zilean,Support,Mage
Zoe,Mage,Support


In [ ]:
flat_champions_modified = pd.concat([flat_champions[:], tags2[:]], axis=1)
flat_champions_modified

,type,format,version,data_version,data_id,data_key,data_name,data_title,data_blurb,data_image,data_tags,data_partype,data_info_attack,data_info_defense,data_info_magic,data_info_difficulty,data_stats_hp,data_stats_hpperlevel,data_stats_mp,data_stats_mpperlevel,data_stats_movespeed,data_stats_armor,data_stats_armorperlevel,data_stats_spellblock,data_stats_spellblockperlevel,data_stats_attackrange,data_stats_hpregen,data_stats_hpregenperlevel,data_stats_mpregen,data_stats_mpregenperlevel,data_stats_crit,data_stats_critperlevel,data_stats_attackdamage,data_stats_attackdamageperlevel,data_stats_attackspeedperlevel,data_stats_attackspeed,Champion_class,Champion_class_2
Aatrox,champion,standAloneComplex,10.23.1,10.23.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","[Fighter, Tank]",Blood Well,8,4,3,4,580.0,90.0,0.0,0.0,345.0,38.000,3.25,32.1,1.25,175.0,3.0,1.00,0.000,0.000,0.0,0.0,60.000,5.00,2.500,0.651,Fighter,Tank
Ahri,champion,standAloneComplex,10.23.1,10.23.1,Ahri,103,Ahri,the Nine-Tailed Fox,Innately connected to the latent power of Rune...,"{'full': 'Ahri.png', 'sprite': 'champion0.png'...","[Mage, Assassin]",Mana,3,4,8,5,526.0,92.0,418.0,25.0,330.0,20.880,3.50,30.0,0.50,550.0,5.5,0.60,8.000,0.800,0.0,0.0,53.040,3.00,2.000,0.668,Mage,Assassin
Akali,champion,standAloneComplex,10.23.1,10.23.1,Akali,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,"{'full': 'Akali.png', 'sprite': 'champion0.png...",[Assassin],Energy,5,3,8,7,575.0,95.0,200.0,0.0,345.0,23.000,3.50,37.0,1.25,125.0,8.0,0.50,50.000,0.000,0.0,0.0,62.400,3.30,3.200,0.625,Assassin,NaN
Alistar,champion,standAloneComplex,10.23.1,10.23.1,Alistar,12,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"{'full': 'Alistar.png', 'sprite': 'champion0.p...","[Tank, Support]",Mana,6,9,5,7,600.0,106.0,350.0,40.0,330.0,44.000,3.50,32.1,1.25,125.0,8.5,0.85,8.500,0.800,0.0,0.0,62.000,3.75,2.125,0.625,Tank,Support
Amumu,champion,standAloneComplex,10.23.1,10.23.1,Amumu,32,Amumu,the Sad Mummy,Legend claims that Amumu is a lonely and melan...,"{'full': 'Amumu.png', 'sprite': 'champion0.png...","[Tank, Mage]",Mana,2,6,8,3,615.0,80.0,285.0,60.0,335.0,30.000,3.80,32.1,1.25,125.0,9.0,0.85,7.382,0.525,0.0,0.0,53.380,3.80,2.180,0.736,Tank,Mage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zed,champion,standAloneComplex,10.23.1,10.23.1,Zed,238,Zed,the Master of Shadows,"Utterly ruthless and without mercy, Zed is the...","{'full': 'Zed.png', 'sprite': 'champion4.png',...",[Assassin],Energy,9,2,1,7,584.0,85.0,200.0,0.0,345.0,32.000,3.50,32.1,1.25,125.0,7.0,0.65,50.000,0.000,0.0,0.0,63.000,3.40,3.300,0.651,Assassin,NaN
Ziggs,champion,standAloneComplex,10.23.1,10.23.1,Ziggs,115,Ziggs,the Hexplosives Expert,"With a love of big bombs and short fuses, the ...","{'full': 'Ziggs.png', 'sprite': 'champion4.png...",[Mage],Mana,2,4,9,4,536.0,92.0,480.0,23.5,325.0,21.544,3.30,30.0,0.50,550.0,6.5,0.60,8.000,0.800,0.0,0.0,54.208,3.10,2.000,0.656,Mage,NaN
Zilean,champion,standAloneComplex,10.23.1,10.23.1,Zilean,26,Zilean,the Chronokeeper,"Once a powerful Icathian mage, Zilean became o...","{'full': 'Zilean.png', 'sprite': 'champion4.pn...","[Support, Mage]",Mana,2,5,8,6,504.0,82.0,452.0,50.0,335.0,24.000,3.80,30.0,0.50,550.0,5.5,0.50,11.335,0.800,0.0,0.0,51.640,3.00,2.130,0.625,Support,Mage
Zoe,champion,standAloneComplex,10.23.1,10.23.1,Zoe,142,Zoe,the Aspect of Twilight,"As the embodiment of mischief, imagination, an...","{'full': 'Zoe.png', 'sprite': 'champion5.png',...","[Mage, Support]",Mana,1,7,8,5,560.0,92.0,425.0,25.0,340.0,20.800,3.50,30.0,0.50,550.0,6.5,0.60,8.000,0.650,0.0,0.0,58.000,3.30,2.500,0.625,Mage,Support


In [ ]:
flat_champions_modified.info()

<class 'pandas.core.frame.DataFrame'>
Index: 152 entries, Aatrox to Zyra
Data columns (total 38 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   type                             152 non-null    object 
 1   format                           152 non-null    object 
 2   version                          152 non-null    object 
 3   data_version                     152 non-null    object 
 4   data_id                          152 non-null    object 
 5   data_key                         152 non-null    object 
 6   data_name                        152 non-null    object 
 7   data_title                       152 non-null    object 
 8   data_blurb                       152 non-null    object 
 9   data_image                       152 non-null    object 
 10  data_tags                        152 non-null    object 
 11  data_partype                     152 non-null    object 
 12  data_info_attack     

In [ ]:
name_csv = 'prueba_1_champions_output.csv'

champions_df = convert_csv(flat_champions_modified,name_csv)

## MERGE DE TODO 

In [ ]:
df_1 = merge(flat_champions_modified,champions_summoner_df)
df_1

TypeError: ignored